# Zookeeper A Distributed Coordination Service for Distributed Applications

### 基础理论
* ZooKeeper is a high-performance coordination service for distributed applications. It exposes common services - such as `naming, configuration management, synchronization, and group services` - in a simple interface so you don't have to write them from scratch. You can use it off-the-shelf to implement `consensus, group management, leader election, and presence protocols`.
* <b>Design Goals</b>
    * ZooKeeper is simple. ZooKeeper allows distributed processes to coordinate with each other through a shared hierarchal namespace which is organized similarly to a standard file system. The name space consists of data registers - called znodes - and these are similar to files and directories. ZooKeeper data is kept in-memory.
    * ZooKeeper is replicated. ZooKeeper itself is intended to be replicated over a sets of hosts called an ensemble.
        * The servers that make up the ZooKeeper service must all know about each other. They maintain an in-memory image of state, along with a transaction logs and snapshots in a persistent store. As long as a majority of the servers are available, the ZooKeeper service will be available.
        * Clients connect to a single ZooKeeper server. The client maintains a TCP connection through which it sends requests, gets responses, gets watch events, and sends heart beats. If the TCP connection to the server breaks, the client will connect to a different server.
    * ZooKeeper is ordered. ZooKeeper stamps each update with a number that reflects the order of all ZooKeeper transactions. 
    * ZooKeeper is fast. It is especially fast in "read-dominant" workloads.
* <b>Guarantees</b>
    * Sequential Consistency - Updates from a client will be applied in the order that they were sent.
    * Atomicity - Updates either succeed or fail. No partial results.
    * Single System Image - A client will see the same view of the service regardless of the server that it connects to.
    * Reliability - Once an update has been applied, it will persist from that time forward until a client overwrites the update.
    * Timeliness - The clients view of the system is guaranteed to be up-to-date within a certain time bound.
* <b>Implementation</b>
    * ZooKeeper Components: <img src="../../images/javaee/zkcomponents.jpg" width="400px">
    * ZooKeeper Components shows the high-level components of the ZooKeeper service. With the exception of the request processor, each of the servers that make up the ZooKeeper service replicates its own copy of each of the components.
    * The replicated database is an in-memory database containing the entire data tree. Updates are logged to disk for recoverability, and writes are serialized to disk before they are applied to the in-memory database.
    * Every ZooKeeper server services clients. Clients connect to exactly one server to submit requests. Read requests are serviced from the local replica of each server database. Requests that change the state of the service, write requests, are processed by an agreement protocol.
        * As part of the agreement protocol all write requests from clients are forwarded to a single server, called the leader. The rest of the ZooKeeper servers, called followers, receive message proposals from the leader and agree upon message delivery. The messaging layer takes care of replacing leaders on failures and syncing followers with leaders.
    * ZooKeeper uses a custom atomic messaging protocol. Since the messaging layer is atomic, ZooKeeper can guarantee that the local replicas never diverge.
    * The data stored at each znode in a namespace is read and written atomically. Reads get all the data bytes associated with a znode and a write replaces all the data.
    
****

### 应用实战
* 快速实战，包含基础的集群模式，例如: https://zookeeper.apache.org/doc/r3.5.4-beta/zookeeperStarted.html
    * For replicated mode, a minimum of three servers are required, and it is strongly recommended that you have an odd number of servers. If you only have two servers, then you are in a situation where if one of them fails, there are not enough machines to form a majority quorum. Two servers is inherently less stable than a single server, because there are two single points of failure.
    * Finally, note the two port numbers after each server name: " 2888" and "3888". Peers use the former port to connect to other peers. Such a connection is necessary so that peers can communicate, for example, to agree upon the order of updates. More specifically, a ZooKeeper server uses this port to connect followers to the leader. When a new leader arises, a follower opens a TCP connection to the leader using this port. Because the default leader election also uses TCP, we currently require another port for leader election. This is the second port in the server entry.
* <b>客户端配置</b>: https://zookeeper.apache.org/doc/r3.5.4-beta/zookeeperProgrammers.html#sc_java_client_configuration
* <b>服务端配置</b>: https://zookeeper.apache.org/doc/r3.5.4-beta/zookeeperAdmin.html#sc_configuration
* <b>ZooKeeper Dynamic Reconfiguration</b>: https://zookeeper.apache.org/doc/r3.5.4-beta/zookeeperReconfig.html
* <b>A Simple Watch Client in java</b>: https://zookeeper.apache.org/doc/r3.5.4-beta/javaExample.html
* <b>分布式barriers and producer-consumer queues实现示例（重点是概念思路，然后利用zookeeper做协调器，保证数据一致性）</b>: https://zookeeper.apache.org/doc/r3.5.4-beta/zookeeperTutorial.html
* <b>A Guide to Deployment and Administration</b>: https://zookeeper.apache.org/doc/r3.5.4-beta/zookeeperAdmin.html
* <b>ZooKeeper Quota's Guide</b>: https://zookeeper.apache.org/doc/r3.5.4-beta/zookeeperQuotas.html
* <b>ZooKeeper Observers</b>: https://zookeeper.apache.org/doc/r3.5.4-beta/zookeeperObservers.html
* <b>ZooKeeper JMX</b>: https://zookeeper.apache.org/doc/r3.5.4-beta/zookeeperJMX.html
    * The JMX technology provides a simple, standard way of managing resources such as applications, devices, and services. Because the JMX technology is dynamic, you can use it to monitor and manage resources as they are created, installed and implemented. You can also use the JMX technology to monitor and manage the Java Virtual Machine (Java VM). <i>(https://www.oracle.com/technetwork/java/javase/tech/docs-jsp-135989.html)</i>
* <b>ZooKeeper Recipes and Solutions</b>: 常规应用解决方案https://zookeeper.apache.org/doc/r3.5.4-beta/recipes.html
    * <b>Barriers</b>
        * Client calls the ZooKeeper API's exists() function on the barrier node, with watch set to true.
        * If exists() returns false, the barrier is gone and the client proceeds
        * Else, if exists() returns true, the clients wait for a watch event from ZooKeeper for the barrier node.
        * When the watch event is triggered, the client reissues the exists() call, again waiting until the barrier node is removed.
    * <b>Queues</b>
        * First designate a znode to hold the queue, the queue node.
        * The distributed clients put something into the queue by calling create() with a pathname ending in "queue-", with the sequence and ephemeral flags in the create() call set to true.
        * A client that wants to be removed from the queue calls ZooKeeper's getChildren() function, with watch set to true on the queue node, and begins processing nodes with the lowest number.
        * The client does not need to issue another getChildren() until it exhausts the list obtained from the first getChildren() call. If there are are no children in the queue node, the reader waits for a watch notification to check the queue again.
    * <b>Locks</b>
        * Call create() with a pathname of "_locknode_/guid-lock-" and the sequence and ephemeral flags set. The guid is needed in case the create() result is missed.
        * Call getChildren() on the lock node without setting the watch flag
        * If the pathname created in step 1 has the lowest sequence number suffix, the client has the lock and the client exits the protocol.
        * The client calls exists() with the watch flag set on the path in the lock directory with the next lowest sequence number.
        * if exists() returns false, go to step 2. Otherwise, wait for a notification for the pathname from the previous step before going to step 2.
    * <b>Two-phased Commit</b>:
        * A coordinator create a transaction node, say "/app/Tx", and one child node per participating site, say "/app/Tx/s_i". When coordinator creates the child node, it leaves the content undefined. 
        * Once each site involved in the transaction receives the transaction from the coordinator, the site reads each child node and sets a watch.
        * Each site then processes the query and votes "commit" or "abort" by writing to its respective node.
        * Once the write completes, the other sites are notified, and as soon as all sites have all votes, they can decide either "abort" or "commit". 
    * <b>Leader Election</b>:
        * The idea is to have a znode, say "/election", 
        * Each znode creates a child znode "/election/guid-n_" with both flags SEQUENCE|EPHEMERAL. 
        * With the sequence flag, ZooKeeper automatically appends a sequence number that is greater than any one previously appended to a child of "/election". 
        * The process that created the znode with the smallest appended sequence number is the leader.
        * Other node is sufficient to watch for the next znode down on the sequence of znodes. 
****

### 设计实现
* <b>Data model and the hierarchical namespace</b>
    * <img src="../../images/javaee/zknamespace.jpg" width="400px">
    * The name space provided by ZooKeeper is much like that of a standard file system. A name is a sequence of path elements separated by a slash (/). Every node in ZooKeeper's name space is identified by a path.
* <b>ZooKeeper nodes</b>
    * <b>Znodes</b>: Every node in a ZooKeeper tree is referred to as a znode. Znodes maintain a `stat structure` that includes version numbers for data changes, acl changes. The stat structure also has timestamps. The version number, together with the timestamp, allows ZooKeeper to validate the cache and to coordinate updates.
        * Each time a znode's data changes, the version number increases. For instance, whenever a client retrieves data, it also receives the version of the data. And when a client performs an update or a delete, it must supply the version of the data of the znode it is changing. If the version it supplies doesn't match the actual version of the data, the update will fail.
        * <b>ZooKeeper Stat Structure</b>:
            * <i>czxid</i>: The zxid of the change that caused this znode to be created.
            * <i>mzxid</i>: The zxid of the change that last modified this znode.
            * <i>pzxid</i>: The zxid of the change that last modified children of this znode.
            * <i>ctime</i>: The time in milliseconds from epoch when this znode was created.
            * <i>mtime</i>: The time in milliseconds from epoch when this znode was last modified.
            * <i>version</i>: The number of changes to the data of this znode.
            * <i>cversion</i>: The number of changes to the children of this znode.
            * <i>aversion</i>: The number of changes to the ACL of this znode.
            * <i>ephemeralOwner</i>: The session id of the owner of this znode if the znode is an ephemeral node. If it is not an ephemeral node, it will be zero.
            * <i>dataLength</i>: The length of the data field of this znode.
            * <i>numChildren</i>: The number of children of this znode.
    * <b>Ephemeral Nodes</b>: ZooKeeper also has the notion of ephemeral nodes. These znodes exists as long as the session that created the znode is active. When the session ends the znode is deleted. 
    * <b>Sequence Nodes -- Unique Naming</b>: When creating a znode you can also request that ZooKeeper append a monotonically increasing counter to the end of path. This counter is unique to the parent znode. The counter has a format of %010d -- that is 10 digits with 0 (zero) padding (the counter is formatted in this way to simplify sorting), i.e. "[path]0000000001". 
    * <b>Container Nodes</b>: ZooKeeper has the notion of container znodes. Container znodes are special purpose znodes useful for recipes such as leader, lock, etc. When the last child of a container is deleted, the container becomes a candidate to be deleted by the server at some point in the future.
    * <b>TTL Nodes</b>: When creating PERSISTENT or PERSISTENT_SEQUENTIAL znodes, you can optionally set a TTL in milliseconds for the znode. If the znode is not modified within the TTL and has no children it will become a candidate to be deleted by the server at some point in the future.
* <b>ZooKeeper Watches</b>: Clients can set a watch on a znode. A watch will be triggered and removed when the znode changes. When a watch is triggered, the client receives a packet saying that the znode has changed
    * `ZooKeeper's definition of a watch: a watch event is one-time trigger, sent to the client that set the watch, which occurs when the data for which the watch was set changes. There are three key points to consider in this definition of a watch:`
        * <b>One-time trigger</b>: One watch event will be sent to the client when the data has changed. For example, if a client does a getData("/znode1", true) and later the data for /znode1 is changed or deleted, the client will get a watch event for /znode1. If /znode1 changes again, no watch event will be sent unless the client has done another read that sets a new watch.
        * <b>Sent to the client</b>: This implies that an event is on the way to the client, but may not reach the client before the successful return code to the change operation reaches the client that initiated the change. Watches are sent asynchronously to watchers. ZooKeeper provides an ordering guarantee: a client will never see a change for which it has set a watch until it first sees the watch event.
        * <b>The data for which the watch was set</b>: It helps to think of ZooKeeper as maintaining two lists of watches: <b>data watches</b> and <b>child watches</b>. 
            * `getData() and exists()` set <b>data watches</b>: `getData() and exists()` return information about the data of the node
            * `getChildren()` sets <b>child watches</b>: `getChildren()` returns a list of children. 
            * `setData()` will trigger <b>data watches</b> for the znode being set 
            * `create()` will trigger a <b>data watch</b> for the znode being created and a <b>child watch</b> for the parent znode
            * `delete()` will trigger both a <b>data watch</b> and a <b>child watch</b> for a znode being deleted as well as a <b>child watch</b> for the parent znode.
    * <i>Watches are maintained locally at the ZooKeeper server to which the client is connected. This allows watches to be lightweight to set, maintain, and dispatch. When a client connects to a new server, the watch will be triggered for any session events. Watches will not be received while disconnected from a server. When a client reconnects, any previously registered watches will be reregistered and triggered if needed.</i>
    * <b>Semantics of Watches</b>
        * Created event: Enabled with a call to exists.
        * Deleted event: Enabled with a call to exists, getData, and getChildren.
        * Changed event: Enabled with a call to exists and getData.
        * Child event: Enabled with a call to getChildren.
    * <b>Things to Remember about Watches</b>
        * Watches are one time triggers; if you get a watch event and you want to get notified of future changes, you must set another watch.
        * Because watches are one time triggers and there is latency between getting the event and sending a new request to get a watch you cannot reliably see every change that happens to a node in ZooKeeper. Be prepared to handle the case where the znode changes multiple times between getting the event and setting the watch again.
        * A watch object, or function/context pair, will only be triggered once for a given notification. For example, if the same watch object is registered for an exists and a getData call for the same file and that file is then deleted, the watch object would only be invoked once with the deletion notification for the file.
        * When you disconnect from a server, you will not get any watches until the connection is reestablished. For this reason session events are sent to all outstanding watch handlers. Use session events to go into a safe mode: you will not be receiving events while disconnected, so your process should act conservatively in that mode.
* <b>ZooKeeper access control using ACLs</b>: The ACL implementation is quite similar to UNIX file access permissions: it employs permission bits to allow/disallow various operations against a node and the scope to which the bits apply. 
    * An ACL specifies sets of ids and permissions that are associated with those ids.
    * An ACL pertains only to a specific znode. In particular it does not apply to children. ACLs are not recursive.
    * <b>ACL Permissions</b>
        * CREATE: you can create a child node
        * READ: you can get data from a node and list its children.
        * WRITE: you can set data for a node
        * DELETE: you can delete a child node
        * ADMIN: you can set permissions
    * <b>Builtin ACL Schemes</b>:
        * Ids are specified using the form `scheme:expression`, where scheme is the authentication scheme that the id corresponds to. The set of valid expressions are defined by the scheme. 
        * ACLs are made up of pairs of `scheme:expression, perms`. The format of the expression is specific to the scheme. 
* <b>ZooKeeper Sessions</b>: A ZooKeeper client establishes a session with the ZooKeeper service by creating a handle to the service using a language binding. Once created, the handle starts of in the CONNECTING state and the client library tries to connect to one of the servers that make up the ZooKeeper service at which point it switches to the CONNECTED state. During normal operation will be in one of these two states.
    * <b>Session status</b>: <img src="../../images/javaee/state_dia.jpg" width="400px">
    * <i>An optional "chroot" suffix may also be appended to the connection string. This will run the client commands while interpreting all paths relative to this root. If used the example would look like: "127.0.0.1:4545/app/a" where the client would be rooted at "/app/a" and all paths would be relative to this root - ie getting/setting/etc... "/foo/bar" would result in operations being run on "/app/a/foo/bar".</i>
    * <b>Session create steps</b>: `(1)` When a client gets a handle to the ZooKeeper service, ZooKeeper creates a ZooKeeper session, represented as a 64-bit number, that it assigns to the client. `(2)` If the client connects to a different ZooKeeper server, it will send the session id as a part of the connection handshake. `(3)` As a security measure, the server creates a password for the session id that any ZooKeeper server can validate. The password is sent to the client with the session id when the client establishes the session. The client sends this password with the session id whenever it reestablishes the session with a new server.
    * <b>Session timeout</b>: 
        * One of the parameters to the ZooKeeper client library call to create a ZooKeeper session is the session timeout in milliseconds. The client sends a requested timeout, the server responds with the timeout that it can give the client. The current implementation requires that the timeout be a minimum of 2 times the tickTime (as set in the server configuration) and a maximum of 20 times the tickTime. The ZooKeeper client API allows access to the negotiated timeout.
        * When a client (session) becomes partitioned from the ZK serving cluster it will begin searching the list of servers that were specified during session creation. Eventually, when connectivity between the client and at least one of the servers is re-established, the session will either again transition to the "connected" state (if reconnected within the session timeout value) or it will transition to the "expired" state (if reconnected after the session timeout). The ZK client library will handle reconnect for you. Only create a new session when you are notified of session expiration (mandatory).
        * Session expiration is managed by the ZooKeeper cluster itself, not by the client. When the ZK client establishes a session with the cluster it provides a "timeout" value detailed above. This value is used by the cluster to determine when the client's session expires. Expirations happens when the cluster does not hear from the client within the specified session timeout period (i.e. no heartbeat). At session expiration the cluster will delete any/all ephemeral nodes owned by that session and immediately notify any/all connected clients of the change (anyone watching those znodes). At this point the client of the expired session is still disconnected from the cluster, it will not be notified of the session expiration until/unless it is able to re-establish a connection to the cluster. The client will stay in disconnected state until the TCP connection is re-established with the cluster, at which point the watcher of the expired session will receive the "session expired" notification.
    * <b>Session wather</b>: Another parameter to the ZooKeeper session establishment call is the default watcher. Watchers are notified when any state change occurs in the client. For example if the client loses connectivity to the server the client will be notified
* <b>Time in ZooKeeper</b>
    * <b>Zxid</b>: Every change to the ZooKeeper state receives a stamp in the form of a zxid (ZooKeeper Transaction Id). This exposes the total ordering of all changes to ZooKeeper. Each change will have a unique zxid and if zxid1 is smaller than zxid2 then zxid1 happened before zxid2.
    * <b>Version numbers</b>: Every change to a node will cause an increase to one of the version numbers of that node. 
        * <i>version</i>: number of changes to the data of a znode
        * <i>cversion</i>: number of changes to the children of a znode
        * <i>aversion</i>: number of changes to the ACL of a znode
    * <b>Ticks</b>: When using multi-server ZooKeeper, servers use ticks to define timing of events such as status uploads, session timeouts, connection timeouts between peers, etc. The tick time is only indirectly exposed through the minimum session timeout (2 times the tick time).
    * <b>Real time</b>: ZooKeeper doesn't use real time, or clock time, at all except to put timestamps into the stat structure on znode creation and znode modification.

****

# ETCD 

### 基础理论 RAFT
* etcd is a distributed key-value store designed to reliably and quickly preserve and provide access to critical data. It enables reliable distributed coordination through distributed locking, leader elections, and write barriers. An etcd cluster is intended for high availability and permanent data storage and retrieval.
* <b>why etcd</b>: https://github.com/etcd-io/etcd/blob/master/Documentation/learning/why.md
    * Etcd supports a wide range of languages and frameworks 
    * Dynamic cluster membership reconfiguration
    * Stable read/write under high load
    * A multi-version concurrency control data model
    * Reliable key monitoring which never silently drop events
    * Lease primitives decoupling connections from sessions
    * APIs for safe distributed shared locks
* <b>Data model</b>: etcd is designed to reliably store infrequently updated data and provide reliable watch queries. etcd exposes previous versions of key-value pairs to support inexpensive snapshots and watch history events. A persistent, multi-version, concurrency-control data model is a good fit for these use cases.
    * etcd stores data in a multiversion persistent key-value store. The persistent key-value store preserves the previous version of a key-value pair when its value is superseded with new data. 
    * <b><i>`The key-value store is effectively immutable; its operations do not update the structure in-place, but instead always generate a new updated structure.`</i></b>
    * All past versions of keys are still accessible and watchable after modification. 
    * To prevent the data store from growing indefinitely over time and from maintaining old versions, the store may be compacted to shed the oldest versions of superseded data.
    * <b>Logical view</b>: The store’s logical view is a flat binary key space. The key space has a lexically sorted index on byte string keys so range queries are inexpensive.
        * <b><i>`Each modification of cluster state, which may change multiple keys, is assigned a global unique ID, called a revision in etcd, from a monotonically increasing counter for reasoning over ordering.`</i></b>
        * <b><i>`The key space maintains multiple revisions. Each atomic mutative operation creates a new revision on the key space.`</i></b>
        * All data held by previous revisions remains unchanged. Old versions of key can still be accessed through previous revisions. 
        * Revisions are indexed as well; ranging over revisions with watchers is efficient. 
        * If the store is compacted to save space, revisions before the compact revision will be removed. Revisions are monotonically increasing over the lifetime of a cluster.
        * A key's life spans a generation, from creation to deletion. Each key may have one or multiple generations. 
        * Creating a key increments the version of that key, starting at 1 if the key does not exist at the current revision. 
        * Deleting a key generates a key tombstone, concluding the key’s current generation by resetting its version to 0. 
        * Each modification of a key increments its version; so, versions are monotonically increasing within a key's generation. 
        * Once a compaction happens, any generation ended before the compaction revision will be removed, and values set before the compaction revision except the latest one will be removed.
    * <b>Physical view</b>: etcd stores the physical data as key-value pairs in a persistent b+tree. Each revision of the store’s state only contains the delta from its previous revision to be efficient. A single revision may correspond to multiple keys in the tree.
        * <b><i>`The key of key-value pair is a 3-tuple (major, sub, type). Major is the store revision holding the key. Sub differentiates among keys within the same revision. Type is an optional suffix for special value (e.g., t if the value contains a tombstone).`</i></b> 
        * <b><i>`The value of the key-value pair contains the modification from previous revision, thus one delta from previous revision.`</i></b>
        * The b+tree is ordered by key in lexical byte-order. Ranged lookups over revision deltas are fast; this enables quickly finding modifications from one specific revision to another. Compaction removes out-of-date keys-value pairs.
        * <b><i>`etcd also keeps a secondary in-memory btree index to speed up range queries over keys. The keys in the btree index are the keys of the store exposed to user. The value is a pointer to the modification of the persistent b+tree. Compaction removes dead pointers.`</i></b>
* <b>etcd3 API</b>: https://github.com/etcd-io/etcd/blob/master/Documentation/learning/api.md
    * <b>Revisions</b>
        * etcd maintains a 64-bit cluster-wide counter, the store revision, that is incremented each time the key space is modified. 
        * The revision serves as a global logical clock, sequentially ordering all updates to the store. The change represented by a new revision is incremental; the data associated with a revision is the data that changed the store. <i>Internally, a new revision means writing the changes to the backend's B+tree, keyed by the incremented revision.</i>
        * Revisions become more valuable when considering etcd3's multi-version concurrency control backend. The MVCC model means that the key-value store can be viewed from past revisions since historical key revisions are retained. 
        * The retention policy for this history can be configured by cluster administrators for fine-grained storage management; usually etcd3 discards old revisions of keys on a timer. A typical etcd3 cluster retains superseded key data for hours. 
        * This also provides reliable handling for long client disconnection, not just transient network disruptions: watchers simply resume from the last observed historical revision. Similarly, to read from the store at a particular point-in-time, read requests can be tagged with a revision to return keys from a view of the key space at the point-in-time that revision was committed.
* <b>Glossary</b>: https://github.com/etcd-io/etcd/blob/master/Documentation/learning/glossary.md

****

### 应用实战
* `Distributed systems use etcd as a consistent key-value store for configuration management, service discovery, and coordinating distributed work.`
* 所有应用场景操作示例: https://github.com/etcd-io/etcd/blob/master/Documentation/demo.md
* 本地基本操作示例以及详解: 
    * https://github.com/etcd-io/etcd/blob/master/Documentation/docs.md#developing-with-etcd
    * https://github.com/etcd-io/etcd/blob/master/Documentation/dev-guide/local_cluster.md
    * https://github.com/etcd-io/etcd/blob/master/Documentation/dev-guide/interacting_v3.md
* 集群基本操作示例以及详解: 
    * https://github.com/etcd-io/etcd/blob/master/Documentation/docs.md#operating-etcd-clusters
    * https://github.com/etcd-io/etcd/blob/master/Documentation/op-guide/configuration.md
* API规范:
    * https://github.com/etcd-io/etcd/blob/master/Documentation/dev-guide/api_reference_v3.md
    * https://github.com/etcd-io/etcd/blob/master/Documentation/dev-guide/api_concurrency_reference_v3.md
* Client集成: https://github.com/etcd-io/etcd/blob/master/Documentation/integrations.md

****

# Consul: 商业集成
* https://www.consul.io/docs/internals/architecture.html
* Consul is an end-to-end service discovery framework. It provides built-in health checking, failure detection, and DNS services.
* If looking for end-to-end cluster service discovery, etcd will not have enough features; choose Kubernetes, Consul, or SmartStack.